In [2]:
import pandas as pd
import numpy as np
import json
import re
from difflib import SequenceMatcher
from src.ingestion import download_from_drive
from tqdm.notebook import tqdm  # For progress bars

In [4]:
# Load AIDA & Cleaning 
df_s_anagrafica = download_from_drive("csv_startup_anagrafica")

# Settare index su colonna Unamed: 0
df_s_anagrafica.set_index("Unnamed: 0", inplace=True)

# Rinominare colonna Unnamed: 0 in Index
df_s_anagrafica.index.name = "Index"

# Rinominare colonna ATECO 2007 codice in ATECO
df_s_anagrafica.rename(columns={"ATECO 2007\ncodice": "ATECO"}, inplace=True)

## Data Cleaning

# Remove rows where ATECO is empty or NaN
df_s_anagrafica.dropna(subset=['ATECO'], inplace=True)

# Remove column Fondazione (beacuse is empty)
df_s_anagrafica.drop(columns=['Fondazione'], inplace=True)

# Convert ATECO in integer
df_s_anagrafica['ATECO'] = df_s_anagrafica['ATECO'].astype(int)

# Convert the column where in the title there is "Codice Postale" in integer
df_s_anagrafica.loc[:, df_s_anagrafica.columns.str.contains('Codice postale')] = df_s_anagrafica.loc[:, df_s_anagrafica.columns.str.contains('Codice postale')].astype('Int64')

# Convert the index column to integer
df_s_anagrafica.index = df_s_anagrafica.index.astype(int)

# Convert column Partita IVA to see entirely
df_s_anagrafica["Partita IVA"] = df_s_anagrafica["Partita IVA"].apply(lambda x: '{:.0f}'.format(x))
df_s_anagrafica["Partita IVA"]

# if Sede operativa - Provincia is NaN, insert the value contained in Provincia
df_s_anagrafica['Sede operativa - Provincia'].fillna(df_s_anagrafica['Provincia'], inplace=True)

# if Sede operativa - Codice postale is NaN or <NA>, insert the value contained in Indirizzo sede legale - Codice postale
df_s_anagrafica['Sede operativa - Codice postale'].fillna(df_s_anagrafica['Indirizzo sede legale - Codice postale'], inplace=True)

# if Sede operativa - Regione - Regione is NaN, insert the value contained in Indirizzo sede legale - Regione
df_s_anagrafica['Sede operativa - Regione - Regione'].fillna(df_s_anagrafica['Indirizzo sede legale - Regione'], inplace=True)

df_anagrafica=df_s_anagrafica

/tmp/ipykernel_11408/563454185.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<IntegerArray>
[98066, 20121, 80122, 98051, 24125, 90135, 80143,  6012,  8011, 72017,
 ...
 15076, 15121, 15121, 92013, 92019, 92026, 92100,  <NA>, 92100, 92100]
Length: 11954, dtype: Int64' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_s_anagrafica.loc[:, df_s_anagrafica.columns.str.contains('Codice postale')] = df_s_anagrafica.loc[:, df_s_anagrafica.columns.str.contains('Codice postale')].astype('Int64')
/tmp/ipykernel_11408/563454185.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<IntegerArray>
[ <NA>, 20121, 80144,  <NA>,  <NA>, 90135, 80100,  <NA>,  8100, 72017,
 ...
  <NA>,  <NA>,  <NA>,  <NA>,  <NA>,  <NA>,  <NA>,  <NA>,  <NA>,  <NA>]
Length: 11954, dtype: Int64' has dtype incompatible with float64, pl

In [12]:
df_s_anagrafica.head()

,Ragione sociale,BvD ID number,Codice fiscale,Numero CCIAA,Partita IVA,Indirizzo sede legale,Provincia,Indirizzo sede legale - Codice postale,Indirizzo sede legale - Regione,Sede operativa,Sede operativa - Provincia,Sede operativa - Codice postale,Sede operativa - Regione - Regione,ATECO,Anno di costituzione,Stato giuridico,Forma giuridica
Index,,,,,,,,,,,,,,,,,
1,RAINSHOW S.R.L.,IT03595550835,3595550835,ME0248323,3595550835,"VIA FRANCESCO CRISPI, 4",Messina,98066,Sicilia,NaN,Messina,98066,Sicilia,711210,06/12/2019,Attiva,S.R.L.
2,CHECKSIG S.R.L. SOCIETA' BENEFIT,IT11028330964,11028330964,MI2573724,11028330964,"PZA DEL LIBERTY, 8",Milano,20121,Lombardia,PIAZZA DEL LIBERTY 8,Milano,20121,LOMBARDIA,620100,25/10/2019,Attiva,S.R.L.
3,OTOFARMA S.P.A.,IT08431301210,8431301210,NA0957454,8431301210,"VLE ANTONIO GRAMSCI, 17/B",Napoli,80122,Campania,VIA VIC S.M. DEL PIANTO TORRE 2,Napoli,80144,CAMPANIA,266002,13/07/2016,Attiva,S.P.A.
4,SIKELIA OIL S.R.L.,IT03577640836,3577640836,ME0247028,3577640836,"VIA STRETTO I COCCOMELLI, 64",Messina,98051,Sicilia,NaN,Messina,98051,Sicilia,473000,15/07/2019,Attiva,S.R.L.
5,POWER.IT S.R.L.,IT04355290166,4355290166,BG0456683,4355290166,"VIA ALESSANDRO NOLI, 13",Bergamo,24125,Lombardia,NaN,Bergamo,24125,Lombardia,352300,31/01/2019,Attiva,S.R.L.


In [19]:
# Load CRUNCHBASE & Cleaning
df_crunchbase = download_from_drive("estrazione_crunchbase_5k")


In [23]:
df_crunchbase.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 56 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   Organization Name                    5000 non-null   object
 1   Organization Name URL                5000 non-null   object
 2   IPO Status                           5000 non-null   object
 3   Company Type                         4884 non-null   object
 4   Industries                           4869 non-null   object
 5   Number of Employees                  4575 non-null   object
 6   Headquarters Location                5000 non-null   object
 7   CB Rank (Organization)               4998 non-null   Int64 
 8   CB Rank (Company)                    4998 non-null   Int64 
 9   Similar Companies                    4506 non-null   Int64 
 10  Number of Events                     41 non-null     Int64 
 11  IPO Date                             11 non

In [ ]:
# Delete "Stage", "Stock Symbol", "Stock Symbol URL", "CB Rank (School)", "Stock Exchange", "Valuation at IPO", Valuation at IPO Currency, Valuation at IPO (in USD), Money Raised at IPO, Money Raised at IPO Currency, Money Raised at IPO (in USD), Delisted Date, Delisted Date Precision, Acquisition Terms, Price, Price Currency, Price (in USD), Number of Acquisitions, Accelerator Program Type, Accelerator Duration (in weeks), Accelerator Application Deadline, Investment Stage, Investor Type, Number of Portfolio Organizations columns
df_crunchbase.drop(columns=['Stage', 'Stock Symbol', 'Stock Symbol URL', 'CB Rank (School)', 'Stock Exchange', 'Valuation at IPO', 'Valuation at IPO Currency', 'Valuation at IPO (in USD)', 'Money Raised at IPO', 'Money Raised at IPO Currency', 'Money Raised at IPO (in USD)', 'Delisted Date', 'Delisted Date Precision', 'Acquisition Terms', 'Price', 'Price Currency', 'Price (in USD)', 'Number of Acquisitions', 'Accelerator Program Type', 'Accelerator Duration (in weeks)', 'Accelerator Application Deadline', 'Investment Stage', 'Investor Type', 'Number of Portfolio Organizations'], inplace=True)

# Convert in integer the column CB Rank (Organization), CB Rank (Company), Similar Companies, Number of Events, Number of Investors, Number of Lead Investors, Total Funding Amount, Total Funding Amount (in USD), Last Equity Funding Amount, Last Equity Funding Amount (in USD), Last Funding Amount, Last Funding Amount (in USD), Number of Funding Rounds, Number of Founders, Postal Code
# Remove commas and convert to integers
df_crunchbase['CB Rank (Organization)'] = df_crunchbase['CB Rank (Organization)'].str.replace(',', '').astype('Int64')
df_crunchbase['CB Rank (Company)'] = df_crunchbase['CB Rank (Company)'].str.replace(',', '').astype('Int64')
df_crunchbase['Similar Companies'] = df_crunchbase['Similar Companies'].astype('Int64')
df_crunchbase['Number of Events'] = df_crunchbase['Number of Events'].astype('Int64')
df_crunchbase['Number of Investors'] = df_crunchbase['Number of Investors'].astype('Int64')
df_crunchbase['Number of Lead Investors'] = df_crunchbase['Number of Lead Investors'].astype('Int64')
df_crunchbase['Total Funding Amount'] = df_crunchbase['Total Funding Amount'].astype('Int64')
df_crunchbase['Total Funding Amount (in USD)'] = df_crunchbase['Total Funding Amount (in USD)'].astype('Int64')
df_crunchbase['Last Equity Funding Amount'] = df_crunchbase['Last Equity Funding Amount'].astype('Int64')
df_crunchbase['Last Equity Funding Amount (in USD)'] = df_crunchbase['Last Equity Funding Amount (in USD)'].astype('Int64')
df_crunchbase['Last Funding Amount'] = df_crunchbase['Last Funding Amount'].astype('Int64')
df_crunchbase['Last Funding Amount (in USD)'] = df_crunchbase['Last Funding Amount (in USD)'].astype('Int64')
df_crunchbase['Number of Funding Rounds'] = df_crunchbase['Number of Funding Rounds'].astype('Int64')
df_crunchbase['Number of Founders'] = df_crunchbase['Number of Founders'].astype('Int64')

# Clean the Postal Code column by removing non-numeric characters
df_crunchbase['Postal Code'] = df_crunchbase['Postal Code'].str.extract('(\d+)')  # Extract only numeric parts
df_crunchbase['Postal Code'] = df_crunchbase['Postal Code'].astype('Int64')  # Convert to integer

# Delete redondant columns: Total Funding Amount, Total Funding Amount Currency, Last Equity Funding Amount, Last Equity Funding Amount Currency, Last Funding Amount, Last Funding Amount Currency
df_crunchbase.drop(columns=['Total Funding Amount Currency', 'Last Equity Funding Amount Currency', 'Last Funding Amount Currency'], inplace=True)



<>:21: SyntaxWarning: invalid escape sequence '\d'
<>:21: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_11408/1296796625.py:21: SyntaxWarning: invalid escape sequence '\d'
  df_crunchbase['Postal Code'] = df_crunchbase['Postal Code'].str.extract('(\d+)')  # Extract only numeric parts


In [25]:
pd.options.display.max_columns = None  # Show all columns
df_crunchbase.head()

,Organization Name,Organization Name URL,IPO Status,Company Type,Industries,Number of Employees,Headquarters Location,CB Rank (Organization),CB Rank (Company),Similar Companies,Number of Events,IPO Date,Acquisition Type,Announced Date,Announced Date Precision,Acquired by,Acquired by URL,Transaction Name,Transaction Name URL,Acquisition Status,Number of Investors,Number of Lead Investors,Top 5 Investors,Total Funding Amount,Total Funding Amount (in USD),Last Equity Funding Type,Last Equity Funding Amount,Last Equity Funding Amount (in USD),Last Funding Type,Last Funding Amount,Last Funding Amount (in USD),Last Funding Date,Funding Status,Number of Funding Rounds,Founders,Number of Founders,Industry Groups,Full Description,Phone Number,Contact Email,LinkedIn,Website,Closed Date,Closed Date Precision,Exit Date,Exit Date Precision,Founded Date,Founded Date Precision,Operating Status,Description,Estimated Revenue Range,Postal Code,Headquarters Regions
0,Enilive,https://www.crunchbase.com/organization/enilive,Private,For Profit,"Biofuel, Car Sharing, Energy Efficiency, Trans...",1001-5000,"Roma, Lazio, Italy",1495,1173,10,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,Kohlberg Kravis Roberts,3525500000,3794908074,NaN,<NA>,<NA>,Secondary Market,587500000,614910239,2025-02-18,NaN,2,NaN,<NA>,"Energy, Natural Resources, Sustainability, Tra...",Enilive is Eni's mobility transformation compa...,NaN,NaN,https://www.linkedin.com/company/enilive,http://www.enilive.com,NaN,NaN,NaN,NaN,2023-01-01,year,Active,Enilive is Eni's mobility transformation compa...,NaN,144,"European Union (EU), Europe, Middle East, and ..."
1,Electrolux Professional,https://www.crunchbase.com/organization/electr...,Public,For Profit,"Laundry and Dry-cleaning, Machinery Manufacturing",1001-5000,"Pordenone, Friuli-Venezia Giulia, Italy",3453,2808,11,<NA>,2020-05-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Made Acquisitions,<NA>,<NA>,NaN,140000000,137227995,NaN,<NA>,<NA>,Post-IPO Debt,140000000,137227995,2022-09-30,IPO,1,Axel Wenner-Gren,1,"Clothing and Apparel, Manufacturing",Electrolux Professional is a global provider o...,NaN,NaN,https://www.linkedin.com/company/electrolux-pr...,https://www.electroluxprofessional.com,NaN,NaN,2020-05-27,day,2019-01-01,day,Active,Electrolux Professional is a global provider o...,$1B to $10B,<NA>,"European Union (EU), Europe, Middle East, and ..."
2,NanoPhoria,https://www.crunchbase.com/organization/nanoph...,Private,For Profit,"Analytics, Biotechnology, Health Care",1-10,"Milan, Lombardia, Italy",4329,3530,1,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,EIC Accelerator,17500000,18349624,Seed,15000000,15728249,Seed,15000000,15728249,2025-02-17,Seed,2,NaN,<NA>,"Biotechnology, Data and Analytics, Health Care...",NanoPhoria is a biotechnology company focused ...,NaN,info@nanophoria.com,https://www.linkedin.com/company/nanophoria,https://nanophoria.com,NaN,NaN,NaN,NaN,2022-01-01,year,Active,"NanoPhoria develops non-viral, nano-delivery s...",$1M to $10M,20122,"European Union (EU), Europe, Middle East, and ..."
3,Akamas,https://www.crunchbase.com/organization/akamas,Private,For Profit,"Analytics, Application Performance Management,...",11-50,"Milan, Lombardia, Italy",4411,3623,13,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,United Ventures,10000000,10000000,Seed,10000000,10000000,Seed,10000000,10000000,2025-02-11,Seed,1,"Luca Forni, Stefano Doni",2,"Artificial Intelligence (AI), Data and Analyti...",Akamas is the autonomous optimization platform...,NaN,info@akamas.io,https://www.linkedin.com/company/akamaslabs/,https://www.akamas.io/,NaN,NaN,NaN,NaN,2019-01-01,year,Active,Akamas AI safely optimizes application workloa...,$1M to $10M,20158,"European Union (EU), Europe, Middle East, and ..."
4,Arsenale Bioyards,https://www.crunchbase.com/organization/arsena...,Private,For Profit,"Biotechnology, Health Care, Manufacturing",1-10,"Milan, Lombardia, Italy",4546,3719,10,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,2,"CDP Venture Capital, Plug and Play, Planet A V...",1

In [26]:
# Display the first few rows of each dataframe to understand their structure
print("Startup Anagrafica Dataset:")
display(df_anagrafica.head())

print("\nCrunchbase Dataset:")
display(df_crunchbase.head())

Startup Anagrafica Dataset:


,Ragione sociale,BvD ID number,Codice fiscale,Numero CCIAA,Partita IVA,Indirizzo sede legale,Provincia,Indirizzo sede legale - Codice postale,Indirizzo sede legale - Regione,Sede operativa,Sede operativa - Provincia,Sede operativa - Codice postale,Sede operativa - Regione - Regione,ATECO,Anno di costituzione,Stato giuridico,Forma giuridica
Index,,,,,,,,,,,,,,,,,
1,RAINSHOW S.R.L.,IT03595550835,3595550835,ME0248323,3595550835,"VIA FRANCESCO CRISPI, 4",Messina,98066,Sicilia,NaN,Messina,98066,Sicilia,711210,06/12/2019,Attiva,S.R.L.
2,CHECKSIG S.R.L. SOCIETA' BENEFIT,IT11028330964,11028330964,MI2573724,11028330964,"PZA DEL LIBERTY, 8",Milano,20121,Lombardia,PIAZZA DEL LIBERTY 8,Milano,20121,LOMBARDIA,620100,25/10/2019,Attiva,S.R.L.
3,OTOFARMA S.P.A.,IT08431301210,8431301210,NA0957454,8431301210,"VLE ANTONIO GRAMSCI, 17/B",Napoli,80122,Campania,VIA VIC S.M. DEL PIANTO TORRE 2,Napoli,80144,CAMPANIA,266002,13/07/2016,Attiva,S.P.A.
4,SIKELIA OIL S.R.L.,IT03577640836,3577640836,ME0247028,3577640836,"VIA STRETTO I COCCOMELLI, 64",Messina,98051,Sicilia,NaN,Messina,98051,Sicilia,473000,15/07/2019,Attiva,S.R.L.
5,POWER.IT S.R.L.,IT04355290166,4355290166,BG0456683,4355290166,"VIA ALESSANDRO NOLI, 13",Bergamo,24125,Lombardia,NaN,Bergamo,24125,Lombardia,352300,31/01/2019,Attiva,S.R.L.



Crunchbase Dataset:


,Organization Name,Organization Name URL,IPO Status,Company Type,Industries,Number of Employees,Headquarters Location,CB Rank (Organization),CB Rank (Company),Similar Companies,Number of Events,IPO Date,Acquisition Type,Announced Date,Announced Date Precision,Acquired by,Acquired by URL,Transaction Name,Transaction Name URL,Acquisition Status,Number of Investors,Number of Lead Investors,Top 5 Investors,Total Funding Amount,Total Funding Amount (in USD),Last Equity Funding Type,Last Equity Funding Amount,Last Equity Funding Amount (in USD),Last Funding Type,Last Funding Amount,Last Funding Amount (in USD),Last Funding Date,Funding Status,Number of Funding Rounds,Founders,Number of Founders,Industry Groups,Full Description,Phone Number,Contact Email,LinkedIn,Website,Closed Date,Closed Date Precision,Exit Date,Exit Date Precision,Founded Date,Founded Date Precision,Operating Status,Description,Estimated Revenue Range,Postal Code,Headquarters Regions
0,Enilive,https://www.crunchbase.com/organization/enilive,Private,For Profit,"Biofuel, Car Sharing, Energy Efficiency, Trans...",1001-5000,"Roma, Lazio, Italy",1495,1173,10,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,Kohlberg Kravis Roberts,3525500000,3794908074,NaN,<NA>,<NA>,Secondary Market,587500000,614910239,2025-02-18,NaN,2,NaN,<NA>,"Energy, Natural Resources, Sustainability, Tra...",Enilive is Eni's mobility transformation compa...,NaN,NaN,https://www.linkedin.com/company/enilive,http://www.enilive.com,NaN,NaN,NaN,NaN,2023-01-01,year,Active,Enilive is Eni's mobility transformation compa...,NaN,144,"European Union (EU), Europe, Middle East, and ..."
1,Electrolux Professional,https://www.crunchbase.com/organization/electr...,Public,For Profit,"Laundry and Dry-cleaning, Machinery Manufacturing",1001-5000,"Pordenone, Friuli-Venezia Giulia, Italy",3453,2808,11,<NA>,2020-05-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Made Acquisitions,<NA>,<NA>,NaN,140000000,137227995,NaN,<NA>,<NA>,Post-IPO Debt,140000000,137227995,2022-09-30,IPO,1,Axel Wenner-Gren,1,"Clothing and Apparel, Manufacturing",Electrolux Professional is a global provider o...,NaN,NaN,https://www.linkedin.com/company/electrolux-pr...,https://www.electroluxprofessional.com,NaN,NaN,2020-05-27,day,2019-01-01,day,Active,Electrolux Professional is a global provider o...,$1B to $10B,<NA>,"European Union (EU), Europe, Middle East, and ..."
2,NanoPhoria,https://www.crunchbase.com/organization/nanoph...,Private,For Profit,"Analytics, Biotechnology, Health Care",1-10,"Milan, Lombardia, Italy",4329,3530,1,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,EIC Accelerator,17500000,18349624,Seed,15000000,15728249,Seed,15000000,15728249,2025-02-17,Seed,2,NaN,<NA>,"Biotechnology, Data and Analytics, Health Care...",NanoPhoria is a biotechnology company focused ...,NaN,info@nanophoria.com,https://www.linkedin.com/company/nanophoria,https://nanophoria.com,NaN,NaN,NaN,NaN,2022-01-01,year,Active,"NanoPhoria develops non-viral, nano-delivery s...",$1M to $10M,20122,"European Union (EU), Europe, Middle East, and ..."
3,Akamas,https://www.crunchbase.com/organization/akamas,Private,For Profit,"Analytics, Application Performance Management,...",11-50,"Milan, Lombardia, Italy",4411,3623,13,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,United Ventures,10000000,10000000,Seed,10000000,10000000,Seed,10000000,10000000,2025-02-11,Seed,1,"Luca Forni, Stefano Doni",2,"Artificial Intelligence (AI), Data and Analyti...",Akamas is the autonomous optimization platform...,NaN,info@akamas.io,https://www.linkedin.com/company/akamaslabs/,https://www.akamas.io/,NaN,NaN,NaN,NaN,2019-01-01,year,Active,Akamas AI safely optimizes application workloa...,$1M to $10M,20158,"European Union (EU), Europe, Middle East, and ..."
4,Arsenale Bioyards,https://www.crunchbase.com/organization/arsena...,Private,For Profit,"Biotechnology, Health Care, Manufacturing",1-10,"Milan, Lombardia, Italy",4546,3719,10,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,2,"CDP Venture Capital, Plug and Play, Planet A V...",1

In [31]:
df_anagrafica.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11954 entries, 1 to 11967
Data columns (total 17 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   Ragione sociale                         11954 non-null  object
 1   BvD ID number                           11954 non-null  object
 2   Codice fiscale                          11954 non-null  int64 
 3   Numero CCIAA                            11954 non-null  object
 4   Partita IVA                             11954 non-null  object
 5   Indirizzo sede legale                   11954 non-null  object
 6   Provincia                               11942 non-null  object
 7   Indirizzo sede legale - Codice postale  11922 non-null  Int64 
 8   Indirizzo sede legale - Regione         11942 non-null  object
 9   Sede operativa                          2166 non-null   object
 10  Sede operativa - Provincia              11946 non-null  object
 11  Sede op

In [40]:
import pandas as pd
from thefuzz import process, fuzz

def pulisci_stringa(s: str) -> str:
    """
    Esegue operazioni di cleaning basilari su una stringa:
    - conversione a minuscolo
    - rimozione spazi iniziali/finali
    - (eventualmente) rimozione di punteggiatura o caratteri speciali
    """
    if not isinstance(s, str):
        if s is None:
            return ""
        else:
            return str(s)
    s = s.strip().lower()
    # Remove legal entity types
    s = re.sub(r'\b(srl|s\.r\.l|s\.r\.l\.|s.r.l|spa|s\.p\.a|s\.p\.a\.|s.p.a|inc|llc|ltd|gmbh|corp|corporation)\b', '', s)
    
    # Remove special characters and extra spaces
    s = re.sub(r'[^\w\s]', ' ', s)  # Replace special chars with space
    s = re.sub(r'\s+', ' ', s)       # Replace multiple spaces with single space
    return s

def fuzzy_merge(
    df1: pd.DataFrame,
    df2: pd.DataFrame,
    key1: str,
    key2: str,
    threshold: int = 80
) -> pd.DataFrame:
    """
    Esegue un merge 'fuzzy' tra df1 e df2 in base a key1 (df1) e key2 (df2).
    - threshold definisce il valore minimo di similarità (0-100)
    - Ritorna un DataFrame con le corrispondenze migliori trovate per ogni riga di df1.
    """
    # Copia dei DataFrame per non modificare gli originali
    df1_clean = df1.copy()
    df2_clean = df2.copy()
    
    # Creiamo colonne "pulite" per il matching
    df1_clean['key_clean'] = df1_clean[key1].apply(pulisci_stringa)
    df2_clean['key_clean'] = df2_clean[key2].apply(pulisci_stringa)
    
    # Per facilitare le ricerche, creiamo un dizionario {nome_clean: indice} per df2
    unique_keys2 = df2_clean['key_clean'].unique().tolist()
    
    # Lista per salvare i risultati
    risultati = []

    for idx1, row in df1_clean.iterrows():
        name_to_match = row['key_clean']
        
    # Trova il miglior match in df2 usando thefuzz.process
        # extractOne ritorna (best_match, score, indice)
        best_match, best_score = process.extractOne(name_to_match, unique_keys2, scorer=fuzz.partial_ratio)
        
        if best_score >= threshold:
            # Recuperiamo la riga corrispondente in df2
            match_row = df2_clean[df2_clean['key_clean'] == best_match].iloc[0]
            
            # Controlla se il Codice Postale corrisponde
            #if 'Indirizzo sede legale - Codice postale' in df1_clean.columns and 'Postal Code' in df2_clean.columns:
            #    if row.get('Postal Code') != match_row.get('Postal Code'):
            #        continue  # Escludi questa corrispondenza se i Codici Postali non corrispondono
            
            # Costruiamo un dizionario con i dati combinati
            combined_data = {}
            
            # Dati di df1
            for col in df1_clean.columns:
                combined_data[f"df1_{col}"] = row[col]
            
            # Dati di df2
            for col in df2_clean.columns:
                combined_data[f"df2_{col}"] = match_row[col]
            
            combined_data['similarity_score'] = best_score
            
            risultati.append(combined_data)
        else:
            # Se non supera la soglia di similarità, si può comunque salvare la riga di df1 senza match
            # (oppure scartarla del tutto, a seconda dell’obiettivo)
            combined_data = {f"df1_{col}": row[col] for col in df1_clean.columns}
            combined_data['df2_key_clean'] = None
            combined_data['similarity_score'] = best_score
            risultati.append(combined_data)
    
    # Convertiamo la lista di dizionari in DataFrame
    df_risultati = pd.DataFrame(risultati)
    return df_risultati

# Esempio d'uso
# df_merged = fuzzy_merge(df_anagrafica, df_crunchbase, "Ragione sociale", "Organization Name", threshold=80)

In [ ]:
# Applica funzione fuzzy_merge per trovare corrispondenze tra le due tabelle

In [41]:
df_risultati = fuzzy_merge(df_anagrafica, df_crunchbase, "Ragione sociale", "Organization Name", threshold=90)
df_risultati
df_risultati.shape

Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: ' ']


(11954, 73)

In [42]:
df_risultati[df_risultati['similarity_score'] >= 90][['df1_Ragione sociale', 'df2_Organization Name', 'similarity_score']].sort_values(by='similarity_score', ascending=True)

,df1_Ragione sociale,df2_Organization Name,similarity_score
11876,FLOCKERS SOCIETA' A RESPONSABILITA' LIMITATA S...,Semplifico,90
11866,FUKYO MEDIA SOCIETA' A RESPONSABILITA' LIMITAT...,Semplifico,90
11892,ARPA GROUP S.R.L.,B.PHARMA GROUP,90
10046,XTRAVEL SOCIETA' A RESPONSABILITA' LIMITATA SE...,Semplifico,90
10118,MAXDRON UNIPERSONALE SEMPLIFICATA SRL,Semplifico,90
...,...,...,...
29,HIVE S.R.L.,HiveValue,100
2,OTOFARMA S.P.A.,OTO,100
11949,CIFRANDA S.R.L.,Anda,100
11952,PISTACCHIOBAFFO SOCIETA' A RESPONSABILITA' LIM...,TAC,100


In [ ]:
# Create a function to merge the two dataframes



In [35]:
df_risultati.columns

Index(['df1_Ragione sociale', 'df1_BvD ID number', 'df1_Codice fiscale',
       'df1_Numero CCIAA', 'df1_Partita IVA', 'df1_Indirizzo sede legale',
       'df1_Provincia', 'df1_Indirizzo sede legale - Codice postale',
       'df1_Indirizzo sede legale - Regione', 'df1_Sede operativa',
       'df1_Sede operativa - Provincia', 'df1_Sede operativa - Codice postale',
       'df1_Sede operativa - Regione - Regione', 'df1_ATECO',
       'df1_Anno di costituzione', 'df1_Stato giuridico',
       'df1_Forma giuridica', 'df1_key_clean', 'df2_key_clean',
       'similarity_score'],
      dtype='object')